In [332]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re
import glob
import os

In [ ]:
# Get list of school codes and names

In [331]:
org_codes = pd.read_csv("student_raceeth_all.csv", dtype={"Org Code": object}, usecols = ["SCHOOL", "Org Code", "Year"])

In [ ]:
## Make a list of org codes for each year

In [338]:
## This works

def get_school_attendance_indicators(year):
    mask = org_codes["Year"] == year
    code_list = org_codes[mask]
    code_list = list(set(code_list['Org Code'].tolist()))
    
    dfs = []
    for code in code_list:
        year = str(year)
        attendance = 'http://profiles.doe.mass.edu/profiles/student.aspx?orgcode={}&orgtypecode=6&leftNavId=303&fycode={}'
        attendance = attendance.format(code, year)
        print(attendance)
        attendance = pd.read_html(attendance)
        attendance = attendance[1][1]
        attendance[0] = code
        if year in ['2013', '2014', '2015']:
            attendance[7] = year
        else:
            attendance[9] = year
        attend_df = pd.DataFrame(attendance)
        attend_df = attend_df.transpose()
        dfs.append(attend_df)
    return dfs

In [ ]:
## These have run
attendance_2015 = get_school_attendance_indicators(2015)
attendance_2014 = get_school_attendance_indicators(2014)
attendance_2013 = get_school_attendance_indicators(2013)

In [282]:
attend_13_15 = attendance_2015 + attendance_2014 + attendance_2013
attend_df_13_15 = pd.concat(attend_13_15)

In [ ]:
attend_df_13_15

In [284]:
sch = 'http://profiles.doe.mass.edu/profiles/student.aspx?orgcode=00100055&orgtypecode=6&leftNavId=303&fycode=2015'
headers = pd.read_html(sch)
headers = list(headers[1][0])
headers[0] = "Org Code"
headers_13_15 = headers + ["Year"]

In [292]:
attend_df_13_15.columns = headers_13_15
attend_df_13_15 = attend_df_13_15.reset_index()
attend_df_13_15 = attend_df_13_15.set_index('Org Code')
attend_df_13_15.to_csv("student_attendance_2013_2015.csv")

In [303]:
attendance_2010 = get_school_attendance_indicators(2010)

In [ ]:
attendance_2011 = get_school_attendance_indicators(2011)

In [272]:
attendance_2012 = get_school_attendance_indicators(2012)

In [355]:
sch = 'http://profiles.doe.mass.edu/profiles/student.aspx?orgcode=00100055&orgtypecode=6&leftNavId=303&fycode=2012'
headers = pd.read_html(sch)
headers = list(headers[1][0])
headers[0] = "Org Code"
headers_10_12 = headers + ["Year"]

In [ ]:
attend_10_12 = attendance_2012 + attendance_2010
attend_df_10_12 = pd.concat(attend_10_12)
attend_df_10_12.columns = headers_10_12

attend_df_10_12

In [370]:
headers_11 = headers_10_12[0:9]
headers_11[7] = "Year"

In [371]:
attend_df_11 = pd.concat(attendance_2011)
attend_df_11.columns = headers_11

In [373]:
attend_df_11.columns

Index(['Org Code', 'Attendance Rate', 'Average # of days absent',
       'Absent 10 or more days', 'Chronically Absent (10% or more)',
       'In-School Suspension Rate', 'Out-of-School Suspension Rate', 'Year',
       'Retention Rate'],
      dtype='object')

In [ ]:
# Select columns while 2011 gets sorted out
attend_df_11_cut = attend_df_11[['Org Code', 'Attendance Rate', "Average # of days absent", "Absent 10 or more days", "Chronically Absent (10% or more)", "Year", "Retention Rate"]]
attend_df_10_12_cut = attend_df_10_12[['Org Code', 'Attendance Rate', "Average # of days absent", "Absent 10 or more days", "Chronically Absent (10% or more)", "Year", "Retention Rate"]]
attend_df_13_15 = attend_df_13_15.reset_index()
attend_df_13_15_cut = attend_df_13_15[['Org Code', 'Attendance Rate', "Average # of days absent", "Absent 10 or more days", "Chronically Absent (10% or more)", "Year", "Retention Rate"]]

In [384]:
attendance_all = pd.concat([attend_df_11_cut, attend_df_10_12_cut, attend_df_13_15_cut])

In [385]:
attendance_all
attendance_all.reset_index()
attendance_all = attendance_all.set_index('Org Code')
attendance_all.to_csv("student_attendance_all.csv")